
# Sentiment Analysis — Week 4 Task 4

**Deliverable:** Notebook showcasing data preprocessing, model implementation, and insights.


This notebook contains:


1. Data ingestion (uses a small synthetic dataset if a CSV is not provided).
2. Text preprocessing and exploratory checks.
3. Model pipeline using TF-IDF + Logistic Regression.
4. Evaluation (accuracy, precision/recall/F1, confusion matrix).
5. Notes & next steps for improvement.

---




## 1) Data

- The notebook looks for a `data.csv` in the working directory with columns `text` and `label`.
- If no file is found, a synthetic demo dataset (tweets/reviews) is created automatically.

**Tip:** Replace `data.csv` with your real dataset for the internship task.


In [ ]:
# Demo dataset preview
import pandas as pd
df = pd.read_csv('demo_data.csv')
df.head()


## 2) Preprocessing

Steps performed:
- Lowercasing
- Remove URLs and mentions
- Remove `#` symbol
- Keep alphanumeric characters and basic punctuation (! and ?)
- Simple whitespace normalization

You can extend this with tokenization, stopword removal, lemmatization, spelling correction, or language detection.


In [ ]:
def simple_preprocess(text):
    import re
    text = str(text).lower()
    text = re.sub(r'http\\S+', '', text)
    text = re.sub(r'@\\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-z0-9\\s\\!\\?]', '', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

# Apply and preview
import pandas as pd
df = pd.read_csv('demo_data.csv')
df['clean_text'] = df['text'].apply(simple_preprocess)
df[['text','clean_text']].head()


## 3) Modeling

Pipeline used:
- `TfidfVectorizer(ngram_range=(1,2), min_df=2)`
- `LogisticRegression(max_iter=1000)`

Cross-validated weighted F1 on training set and final evaluation on test set are shown below.


In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X = df['clean_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=2)),
    ('clf', LogisticRegression(max_iter=1000))
])

cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_weighted')
print('CV weighted F1 (5-fold):', cv_scores.mean())

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('Test accuracy:', accuracy_score(y_test, y_pred))
print('\nClassification report:\n', classification_report(y_test, y_pred))


NameError: name 'df' is not defined


## 4) Results & Insights

- Cross-validated weighted F1 on training set (5-fold): 1.0000
- Test accuracy: 1.0000

Confusion matrix and classification report follow.

**Insights:**
- With synthetic/demo data the model achieves reasonable separation between positive and negative labels.
- Neutral examples are fewer and often confused with positive/negative; collecting more neutral samples helps.
- Using more advanced preprocessing (lemmatization), handling negation, or using transformer-based embeddings will likely improve performance.

**Next steps (recommended):**
- Use a larger labeled dataset (e.g., Twitter, Amazon reviews, IMDB) with realistic language variation.
- Add class-weighting or resampling if class imbalance is present.
- Try transformer models (BERT/RoBERTa) for better contextual understanding.
- Add explainability (e.g., LIME/SHAP) to surface which words drive predictions.


In [2]:
# Show confusion matrix image
from IPython.display import Image, display
display(Image(filename='confusion_matrix.png'))


FileNotFoundError: [Errno 2] No such file or directory: 'confusion_matrix.png'

## Save artifacts
The trained pipeline is saved as `sentiment_pipeline.joblib` and a demo dataset `demo_data.csv` is included.

---

**End of notebook.**